## Load Census Data

In [4]:
# functions to run R code for fetching census data
import rpy2.robjects as robjects
import pandas as pd
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# functions for cleaning/manipulating data
from lib.data import (
    get_parcels,
    get_du_est,
    add_columns_from_csv,
    add_columns_from_census,
    fix_geoid_dtypes,
    clean_analytic_dataset,
)

# functions for aggregation, calculations/creating new variables
from lib.variables import process_data


# user-defined parameters
class CONFIG:
    CENSUS_YEAR = 2020
    PATH_PARCELS = r"../data/Parcels_1"
    PATH_DU_EST = r"../data/parcels_clean_duest_stu_spjoin_20240625.csv"


if __name__ == "__main__":
    import os

    print(os.getcwd())
    # CENSUS ==================================================================
    robjects.r("source('lib/DataGathering.R')")

    make_acs_table_t_r = robjects.globalenv["make_acs_table_t"]
    make_acs_table_bg_r = robjects.globalenv["make_acs_table_bg"]

    # Convert the R DataFrame to a pandas DataFrame
    with localconverter(robjects.default_converter + pandas2ri.converter):
        make_acs_table_t = robjects.conversion.rpy2py(make_acs_table_t_r)
        make_acs_table_bg = robjects.conversion.rpy2py(make_acs_table_bg_r)

        acs_table_t_r = make_acs_table_t(CONFIG.CENSUS_YEAR)
        acs_table_bg_r = make_acs_table_bg(CONFIG.CENSUS_YEAR)

        acs_table_t = robjects.conversion.rpy2py(acs_table_t_r)
        acs_table_bg = robjects.conversion.rpy2py(acs_table_bg_r)

    # Durham Open/Parcels =====================================================

    durham_open = get_parcels(CONFIG.PATH_PARCELS)
    parcels_clean = get_du_est(CONFIG.PATH_DU_EST)

    # Joins ===================================================================

    base_dataset = add_columns_from_csv(durham_open, parcels_clean)

    # converting geo_ids to integers for joining with census data
    base_dataset["geo_id_t2020"] = fix_geoid_dtypes(base_dataset["geo_id_t2020"])
    base_dataset["geo_id_b2020"] = fix_geoid_dtypes(base_dataset["geo_id_b2020"])
    base_dataset["geo_id_bg2020"] = fix_geoid_dtypes(base_dataset["geo_id_bg2020"])

    # adding columns from census data
    base_dataset = add_columns_from_census(base_dataset, acs_table_t, "t")
    base_dataset = add_columns_from_census(base_dataset, acs_table_bg, "bg")

    # Calculations ============================================================
    base_dataset = clean_analytic_dataset(base_dataset)

    base_dataset = process_data(base_dataset)

    # base_dataset.to_file("../data/dps_data/base_dataset.shp")

    # agg_cols = []

R[write to console]: To install your API key for use in future sessions, run this function with `install = TRUE`.

R[write to console]: Getting data from the 2016-2020 5-year ACS

R[write to console]: Downloading feature geometry from the Census website.  To cache shapefiles for use in future sessions, set `options(tigris_use_cache = TRUE)`.

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'



/Users/dhavalpotdar/Workarea/Data+/DPS/durham-public-schools/src


R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Getting data from the 2016-2020 5-year ACS

R[write to console]: Downloading feature geometry from the Census website.  To cache shapefiles for use in future sessions, set `options(tigris_use_cache = TRUE)`.

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '

In [10]:
base_dataset.head()

,OBJECTID_1,OBJECTID,REID,PIN,PROPERTY_D,LOCATION_A,SPEC_DIST,LAND_CLASS,ACREAGE,PROPERTY_O,OWNER_MAIL,geometry,designation,housing_type,du_est_final,students2324,students2223,students2122,students2021,geo_id_b2010,geo_id_b2020,geo_id_bg2010,geo_id_bg2020,sch_id_base1819_es,sch_id_base_es,sch_id_gt_es,sch_id_yr_es,sch_id_yr_optout_es,sch_id_zone,sch_id_base_hs,sch_id_gt_hs,sch_id_base1819_ms,sch_id_base_ms,sch_id_gt_ms,sch_id_yr_ms,pu_2122_833,pu_2324_848,geo_id_t2010,geo_id_t2020,TOTAL_PROP_VALUE,estimate_rent_total_t,moe_rent_total_t,estimate_median_house_value_t,estimate_median_year_structure_build_t,estimate_housing_units_t,pct_vacant_t,pct_owner_occupied_t,estimate_rent_total_bg,moe_rent_total_bg,estimate_median_house_value_bg,estimate_median_year_structure_build_bg,estimate_housing_units_bg,pct_vacant_bg,pct_owner_occupied_bg,unit_val,unit_val_cat,unit_val_cat_single,unit_val_cat_multi,unit_val_avg_pu2020,unit_val_cat_single_avg_pu2020,unit_val_cat_multi_avg_pu2020,unit_val_avg_b2020,unit_val_cat_single_avg_b2020,unit_val_cat_multi_avg_b2020,unit_val_avg_bg2020,unit_val_cat_single_avg_bg2020,unit_val_cat_multi_avg_bg2020,unit_val_avg_t2020,unit_val_cat_single_avg_t2020,unit_val_cat_multi_avg_t2020,unit_val_avg_b2010,unit_val_cat_single_avg_b2010,unit_val_cat_multi_avg_b2010,unit_val_avg_bg2010,unit_val_cat_single_avg_bg2010,unit_val_cat_multi_avg_bg2010,unit_val_avg_pu2122,unit_val_cat_single_avg_pu2122,unit_val_cat_multi_avg_pu2122,unit_val_avg_t2010,unit_val_cat_single_avg_t2010,unit_val_cat_multi_avg_t2010
0,5293333.0,18719.0,100001,0822537639,THE SUBURBAN L&P CO/BLK:1 5/LT#01 PL00005A-000041,922 LANCASTER ST,CAPITAL FINANCE,RES/ 2-FAMILY,0.16,"WILKERSON, GLORIA MCBROOM",2429 BIVINS RD,"POLYGON ((-78.91321 36.01309, -78.91320 36.013...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,161020.0,1168.0,330.0,225200.0,1966.0,1625.0,0.04,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.0,0.178808,80510.0,1,NaN,2.0,300282.947095,3.0,3.0,156913.3625,3.0,2.0,315639.188452,3.0,3.0,243880.888123,3.0,3,156913.3625,3.0,2.0,314786.247856,3.0,3.0,290092.64375,3.0,3.0,244684.474049,3.0,3.0
1,5293334.0,18271.0,100002,0822537634,THE SUBURBAN L&P CO/BLK:1 5/LT#03 PL00005A-000041,920 LANCASTER ST,CAPITAL FINANCE,RES/ 2-FAMILY,0.16,SCANLON REALTY CORP,117 PARK AVENUE,"POLYGON ((-78.91274 36.01307, -78.91275 36.012...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,180557.0,1168.0,330.0,225200.0,1966.0,1625.0,0.04,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.0,0.178808,90278.5,1,NaN,2.0,300282.947095,3.0,3.0,156913.3625,3.0,2.0,315639.188452,3.0,3.0,243880.888123,3.0,3,156913.3625,3.0,2.0,314786.247856,3.0,3.0,290092.64375,3.0,3.0,244684.474049,3.0,3.0
2,5293335.0,18958.0,100003,0822537539,PROP-PARRISH E J/W DURHAM /BLK:15/LT#05 PL0000...,918 LANCASTER ST,CAPITAL FINANCE,RES/ 2-FAMILY,0.16,RICHARD L SOLES TRUSTEE;JULIE H SOLES TRUSTEE;...,5607 RUSSELL RD,"POLYGON ((-78.91275 36.01293, -78.91276 36.012...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,167187.0,1168.0,330.0,225200.0,1966.0,1625.0,0.04,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.0,0.178808,83593.5,1,NaN,2.0,300282.947095,3.0,3.0,156913.3625,3.0,2.0,315639.188452,3.0,3.0,243880.888123,3.0,3,156913.3625,3.0,2.0,314786.247856,3.0,3.0,290092.64375,3.0,3.0,244684.474049,3.0,3.0
3,5293336.0,17297.0,100004,0822537534,PROP-PARRISH E J/W DURHAM /BLK:15/LT#07 PL0000...,916 LANCASTER ST,CAPITAL FINANCE,RES/ 1-FAMILY,0.16,"LOUKAKIS, IOANNIS",916 LANCASTER ST,"POLYGON ((-78.91276 36.01280, -78.91276 36.012...",single,sf_detach,1,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003

In [35]:
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

import geopandas as gpd
import fiona
import matplotlib.pyplot as plt

gdb_path = "../data/dps_all_layers20240208.gdb"

# List available layers in the GDB
layers = gpd.io.file.fiona.listlayers(gdb_path)
print("Available layers:", layers)

# Read a specific layer from the GDB
layer_name = "bg2020"  # replace with the actual layer name
gdf = gpd.read_file(gdb_path, layer=layer_name)

Available layers: ['ES_base_1819', 'MS_base_1819', 'ES_yr_2223', 'MS_gt_2526', 'MS_base_2223', 'MS_yr_2223', 'HS_gt_2526', 'HS_base_2223', 'b2020', 'bg2020', 't2020', 'addr_durm_20230623', 'parcels_durm_20230627', 'ES_base_2223', 'PU_2223_982', 'ES_gt_2425', 'ES_yr_optout_2223', 'ES_zone_2223', 'Durham', 'PU_2324_848', 'schools_charter_2021', 'Countywide_HS_Magnet_SchoolDistrict_2022_2023', 'Countywide_Secondary_SchoolDistrict_2022_2023', 'schools_private_1920', 'schools_dps_base_23_24_live', 'MS_yr_2223_all_lyr', 'ES_zone_2223_extended', 'ES_yr_optout_2223_extended', 'MS_yr_2223_extended', 'regions_2024_25', 'regions_2025_26', 'parcels_durm_20230627_filled', 'b2010', 'bg2010', 't2010', 'ES_drive1_2223', 'HS_drive1_2223', 'MS_drive1_2223', 'HS_euclid1_2223', 'ES_euclid1_2223', 'MS_euclid1_2223', 'schools_private_euclid_1920', 'schools_charter_euclid_2021']


In [20]:
layer_mapping = {
    # 'dps_all_layers_geo_id': 'base_dataset_geo_id'
    "b2020": "geo_id_b2020",
    "bg2020": "geo_id_bg2020",
    "t2020": "geo_id_t2020",
    "b2010": "geo_id_b2010",
    "bg2010": "geo_id_bg2010",
    "t2010": "geo_id_t2010",
    "PU_2324_848": "pu_2324_848",
}

In [48]:
layer_mapping.values()

dict_values(['geo_id_b2020', 'geo_id_bg2020', 'geo_id_t2020', 'geo_id_b2010', 'geo_id_bg2010', 'geo_id_t2010', 'pu_2324_848'])

In [33]:
aggregations = ["sum", "mean"]

block_group_aggregation_dict = {
    "ACREAGE": aggregations,
    "du_est_final": aggregations,
    "TOTAL_PROP_VALUE": aggregations,
    "unit_val": aggregations,
    # for the census columns, values are already aggregated on the corresponding geog ids,
    # and so, taking the mean will keep the values unchanged
    "estimate_rent_total_bg": "mean",
    "estimate_median_house_value_bg": "mean",
    "estimate_median_year_structure_build_bg": "mean",
    "estimate_housing_units_bg": "mean",
    "pct_vacant_bg": "mean",
    "pct_owner_occupied_bg": "mean",
}

tract_aggregation_dict = {
    "ACREAGE": aggregations,
    "du_est_final": aggregations,
    "TOTAL_PROP_VALUE": aggregations,
    "unit_val": aggregations,
    # for the census columns, values are already aggregated on the corresponding geog ids,
    # and so, taking the mean will keep the values unchanged
    "estimate_rent_total_t": "mean",
    "estimate_median_house_value_t": "mean",
    "estimate_median_year_structure_build_t": "mean",
    "estimate_housing_units_t": "mean",
    "pct_vacant_t": "mean",
    "pct_owner_occupied_t": "mean",
}

In [36]:
geo_layer = "bg2020"


agg_df = (
    base_dataset.groupby(layer_mapping[geo_layer])
    .agg(block_group_aggregation_dict)
    .reset_index()
)

# Flatten the column MultiIndex after aggregation
agg_df.columns = [agg_df.columns[0][0]] + [
    "_".join(col).strip() if type(col) is tuple else col for col in agg_df.columns[1:]
]

agg_df.head()

,geo_id_bg2020,ACREAGE_sum,ACREAGE_mean,du_est_final_sum,du_est_final_mean,TOTAL_PROP_VALUE_sum,TOTAL_PROP_VALUE_mean,unit_val_sum,unit_val_mean,estimate_rent_total_bg_mean,estimate_median_house_value_bg_mean,estimate_median_year_structure_build_bg_mean,estimate_housing_units_bg_mean,pct_vacant_bg_mean,pct_owner_occupied_bg_mean
0,-1,21.22,0.884167,24,1.000000,1027066.0,42794.416667,1.027066e+06,42794.416667,NaN,NaN,NaN,NaN,NaN,NaN
1,370630001011,157.36,0.264916,649,1.092593,121646926.0,204792.804714,1.165756e+08,196255.178872,148.0,188100.0,1948.0,589.0,0.044143,0.737123
2,370630001012,170.88,0.241356,718,1.014124,122649114.0,173233.211864,1.215408e+08,171667.829802,177.0,160700.0,1971.0,859.0,0.000000,0.793946
3,370630001021,123.14,0.375427,965,2.942073,89060053.0,271524.551829,5.543776e+07,169017.571053,529.0,166000.0,1968.0,802.0,0.148379,0.225476
4,370630001022,115.24,0.309785,398,1.069892,79505324.0,213723.989247,7.311520e+07,196546.231183,147.0,224500.0,1950.0,412.0,0.087379,0.609043


In [40]:
merged_gdf = gdf.merge(
    agg_df,
    how="left",
    left_on=layer_mapping[geo_layer],
    right_on=f"{layer_mapping[geo_layer]}",
)

merged_gdf.crs = gdf.crs

In [41]:
merged_gdf.head()

,geo_id_bg2020,geo_nm_bg2020,Shape_Length,Shape_Area,geometry,ACREAGE_sum,ACREAGE_mean,du_est_final_sum,du_est_final_mean,TOTAL_PROP_VALUE_sum,TOTAL_PROP_VALUE_mean,unit_val_sum,unit_val_mean,estimate_rent_total_bg_mean,estimate_median_house_value_bg_mean,estimate_median_year_structure_build_bg_mean,estimate_housing_units_bg_mean,pct_vacant_bg_mean,pct_owner_occupied_bg_mean
0,370630020074,"Block Group 4, Census Tract 20.07, Durham Coun...",18600.013331,1.567234e+07,"MULTIPOLYGON (((2019843.600 804166.596, 201997...",205.85,0.343656,670.0,1.118531,147473708.0,246199.846411,1.390672e+08,232165.578928,130.0,233500.0,1985.0,671.0,0.019374,0.802432
1,370630017052,"Block Group 2, Census Tract 17.05, Durham Coun...",18026.627386,2.012532e+07,"MULTIPOLYGON (((2020838.473 832658.236, 202315...",126.25,0.433849,613.0,2.106529,55297243.0,190024.890034,5.478687e+07,188271.028995,391.0,140100.0,1973.0,600.0,0.033333,0.325862
2,370630010022,"Block Group 2, Census Tract 10.02, Durham Coun...",7622.138470,3.641296e+06,"MULTIPOLYGON (((2035871.399 816172.230, 203607...",55.56,0.219605,383.0,1.513834,34688522.0,137108.782609,2.747822e+07,108609.560606,215.0,111700.0,1952.0,284.0,0.000000,0.242958
3,370630017055,"Block Group 5, Census Tract 17.05, Durham Coun...",16980.364574,1.160153e+07,"MULTIPOLYGON (((2019066.041 832572.212, 201913...",162.43,0.412259,766.0,1.944162,132313744.0,335821.685279,8.844260e+07,224473.604178,185.0,174800.0,1984.0,730.0,0.243836,0.664855
4,370630002002,"Block Group 2, Census Tract 2, Durham County, ...",15532.676727,6.884432e+06,"MULTIPOLYGON (((2028988.142 820911.848, 203130...",92.50,0.281155,497.0,1.510638,110546764.0,336008.401216,9.271954e+07,281822.321806,319.0,276300.0,1946.0,541.0,0.227357,0.236842


In [46]:
# Write the merged GeoDataFrame to a new GDB as a new layer
layer_name = "geo_id_bg2020"
merged_gdf.to_file("../data/outputs/new_db.gdb", layer=layer_name, driver="OpenFileGDB")

In [45]:
fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [ ]:
"students2324",
"students2223",
"students2122",
"students2021",
"geo_id_b2010",
"geo_id_b2020",
"geo_id_bg2010",
"geo_id_bg2020",
"sch_id_base1819_es",
"sch_id_base_es",
"sch_id_gt_es",
"sch_id_yr_es",
"sch_id_yr_optout_es",
"sch_id_zone",
"sch_id_base_hs",
"sch_id_gt_hs",
"sch_id_base1819_ms",
"sch_id_base_ms",
"sch_id_gt_ms",
"sch_id_yr_ms",
"pu_2122_833",
"pu_2324_848",
"geo_id_t2010",
"geo_id_t2020",